# Extending the TrainedModel Table to support Transfer of multiple Stages

In [3]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
if not "stores" in dj.config:
    dj.config["stores"] = {}
dj.config["stores"]["minio"] = {  # store in s3
        "protocol": "s3",
        "endpoint": os.environ.get("MINIO_ENDPOINT", "DUMMY_ENDPOINT"),
        "bucket": "nnfabrik",
        "location": "dj-store",
        "access_key": os.environ.get("MINIO_ACCESS_KEY", "FAKEKEY"),
        "secret_key": os.environ.get("MINIO_SECRET_KEY", "FAKEKEY"),
    }

from nnfabrik.examples import nnfabrik
from nnfabrik.examples.nnfabrik import *
from nnfabrik.templates.transfer import TransferredTrainedModelBase
schema

Schema `nnfabrik_example`

In [4]:
@schema
class TransferredTrainedModel(TransferredTrainedModelBase):
    table_comment = "Transferred trained models"
    nnfabrik = nnfabrik

## (1) Normal Training

### Add some entries to the table (usual step)

In [ ]:
fabrikant_info = dict(fabrikant_name="Your Name", email="your@email.com", affiliation="thelab", dj_username="yourname")
Fabrikant().insert1(fabrikant_info)

In [6]:
Seed().insert([{'seed':7}])

In [7]:
dataset_fn = "nnfabrik.examples.mnist.dataset.mnist_dataset_fn"

dataset_config = dict(batch_size=64) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="A comment about the dataset!");

In [8]:
# specify model function as string (the function must be importable) as well as the model config
model_fn = "nnfabrik.examples.mnist.model.mnist_model_fn"
model_config = dict(h_dim=5) # we specify all the inputs except the ones required by nnfabrik

Model().add_entry(model_fn=model_fn, model_config=model_config, 
                  model_fabrikant="Your Name", model_comment="A comment about the model!");

In [9]:
# specify trainer function as string (the function must be importable) as well as the trainer config
trainer_fn = "nnfabrik.examples.mnist.trainer.mnist_trainer_fn"
trainer_config = dict(epochs=1) # we specify all the inputs except the ones required by nnfabrik

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, 
                  trainer_fabrikant="Your Name", trainer_comment="A comment about the trainer!");

### Populating TrainedModel normally

In [10]:
TransferredTrainedModel.populate()

100%|██████████| 938/938 [00:21<00:00, 43.06it/s]


In [11]:
TransferredTrainedModel()

*transfer_step *model_fn      *model_hash    *dataset_fn    *dataset_hash  *trainer_fn    *trainer_hash  *seed    *collapsed_his *data_transfer comment        score       output     fabrikant_name trainedmodel_t
+------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------+ +------------+ +------------+ +------------+ +---------+ +--------+ +------------+ +------------+
0              nnfabrik.examp 24922759b84307 nnfabrik.examp 9aee736870714f nnfabrik.examp 79e921430b7f44 7                       0              A comment abou 73.7033     =BLOB=     Arne Nix       2021-01-21 16:
 (Total: 1)

Now that we have trained a model on MNIST, we want to transfer it.
For this, we consider two different scenarios, both of which can be fully automated in our transfer framework.

In both cases, the operations boil down to:
* what datasets and/or models and/or trainers you would like to use at a specific transfer step -> specify this in the recipe
* which component should be handed over between two consecutive transfer steps (data and/or model state) -> specify this in the recipe and use an appropriate "trainer"
* which already-exisiting entries you would like this transfer step to be applied on -> specify this in the `populate` restrictions

## (2) Model-state Transfer

### Add the recipes for transfer

First, let's add the dataset that we want to use for the transfer. 

In [12]:
dataset_fn = "nnfabrik.examples.mnist.dataset.mnist_dataset_fn"

dataset_config = dict(batch_size=64, apply_augmentation=True) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="Augmented MNIST");

In [13]:
Dataset()

*dataset_fn    *dataset_hash  dataset_co dataset_fabrik dataset_commen dataset_ts    
+------------+ +------------+ +--------+ +------------+ +------------+ +------------+
nnfabrik.examp 28aefc23085697 =BLOB=     Your Name      Augmented MNIS 2021-01-21 16:
nnfabrik.examp 9aee736870714f =BLOB=     Your Name      A comment abou 2021-01-21 16:
 (Total: 2)

Now, we define the recipe.

In [14]:
transfer_from = {"dataset_fn": 'nnfabrik.examples.mnist.dataset.mnist_dataset_fn', "dataset_hash": '9aee736870714f8b7c3cc084087ce886'}
transfer_to = {"dataset_fn": 'nnfabrik.examples.mnist.dataset.mnist_dataset_fn', "dataset_hash": '28aefc2308569727c6017c66c9122d77'}

In [15]:
DatasetTransferRecipe().add_entry(transfer_from=transfer_from, transfer_to=transfer_to, transfer_step=1)

In [16]:
DatasetTransferRecipe()

*transfer_step *data_transfer *dataset_fn    *dataset_hash  *prev_dataset_ *prev_dataset_
+------------+ +------------+ +------------+ +------------+ +------------+ +------------+
1              0              nnfabrik.examp 28aefc23085697 nnfabrik.examp 9aee736870714f
 (Total: 1)

### Assign the recipes to the table

In [17]:
TransferredTrainedModel.transfer_recipe = [DatasetTransferRecipe()]

### Populate the table for the first transfer step 

In [18]:
TransferredTrainedModel.populate("transfer_step=1")

100%|██████████| 938/938 [00:21<00:00, 43.77it/s]


In [19]:
TransferredTrainedModel().CollapsedHistory()

*collapsed_his *model_fn      *model_hash    *dataset_fn    *dataset_hash  *trainer_fn    *trainer_hash  *seed    *prev_collapse
+------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------+ +------------+
5a8bc548be900e nnfabrik.examp 24922759b84307 nnfabrik.examp 28aefc23085697 nnfabrik.examp 79e921430b7f44 7        b5dfa65c750917
b5dfa65c750917 nnfabrik.examp 24922759b84307 nnfabrik.examp 9aee736870714f nnfabrik.examp 79e921430b7f44 7                      
 (Total: 2)

In [20]:
TransferredTrainedModel()

*transfer_step *model_fn      *model_hash    *dataset_fn    *dataset_hash  *trainer_fn    *trainer_hash  *seed    *collapsed_his *data_transfer comment        score       output     fabrikant_name trainedmodel_t
+------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------+ +------------+ +------------+ +------------+ +---------+ +--------+ +------------+ +------------+
0              nnfabrik.examp 24922759b84307 nnfabrik.examp 9aee736870714f nnfabrik.examp 79e921430b7f44 7                       0              A comment abou 73.7033     =BLOB=     Arne Nix       2021-01-21 16:
1              nnfabrik.examp 24922759b84307 nnfabrik.examp 28aefc23085697 nnfabrik.examp 79e921430b7f44 7        b5dfa65c750917 0              A comment abou 76.1183     =BLOB=     Arne Nix       2021-01-21 16:
 (Total: 2)

## (3) Data Transfer (Knowledge Distillation)

Now that we have seen a simple transfer of the model state between two training runs on similar datasets, let's consider a more challenging scenario.

Assume we want to transfer knowledge between two slightly different models (e.g. with different hidden size) and at the same time have a domain shift in the data. 

### Add the recipes for the intermediate transfer step (dataset generation)

For this step, we keep everything the same, but replace the trainer by a "pseudo-trainer" that simply captures and saves the logits for the whole dataset.

In [21]:
trainer_fn = "nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn"

trainer_config = dict(batch_size=64, apply_augmentation=False) 

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, 
                    trainer_fabrikant="Your Name", trainer_comment="Transfer MNIST Logits");

In [22]:
Trainer()

*trainer_fn    *trainer_hash  trainer_co trainer_fabrik trainer_commen trainer_ts    
+------------+ +------------+ +--------+ +------------+ +------------+ +------------+
nnfabrik.examp 79e921430b7f44 =BLOB=     Your Name      A comment abou 2021-01-21 16:
nnfabrik.examp ab91f734757071 =BLOB=     Your Name      Transfer MNIST 2021-01-21 16:
 (Total: 2)

In [23]:
transfer_from = {"trainer_fn": 'nnfabrik.examples.mnist.trainer.mnist_trainer_fn', "trainer_hash": '79e921430b7f44a205d79d0087b59dc0'}
transfer_to = {"trainer_fn": 'nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn', "trainer_hash": 'ab91f734757071bf0b98ab74c6e8583c'}

In [24]:
TrainerTransferRecipe().add_entry(transfer_from=transfer_from, transfer_to=transfer_to, transfer_step=1, data_transfer=True)

In [25]:
TransferredTrainedModel.transfer_recipe = [TrainerTransferRecipe()]

### Populate the table for the first transfer step 

In [26]:
TransferredTrainedModel.populate("transfer_step=1")

100%|██████████| 938/938 [00:16<00:00, 57.55it/s]


In [27]:
TransferredTrainedModel()

*transfer_step *model_fn      *model_hash    *dataset_fn    *dataset_hash  *trainer_fn    *trainer_hash  *seed    *collapsed_his *data_transfer comment        score       output     fabrikant_name trainedmodel_t
+------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------+ +------------+ +------------+ +------------+ +---------+ +--------+ +------------+ +------------+
0              nnfabrik.examp 24922759b84307 nnfabrik.examp 9aee736870714f nnfabrik.examp 79e921430b7f44 7                       0              A comment abou 73.7033     =BLOB=     Arne Nix       2021-01-21 16:
1              nnfabrik.examp 24922759b84307 nnfabrik.examp 28aefc23085697 nnfabrik.examp 79e921430b7f44 7        b5dfa65c750917 0              A comment abou 76.1183     =BLOB=     Arne Nix       2021-01-21 16:
1              nnfabrik.examp 24922759b84307 nnfabrik.examp 9aee736870714f nnfabrik.examp ab91f734757071 7        b5dfa65c750917 1              Transfer MNIST 0.0         =BLOB=     Arne Nix       2021-01-21 16:
 (Total: 3)

### Add the recipes for the second transfer step (using the generated dataset)

Now we need a dataset that loads the transferred dataset and a trainer that can handle it. Let's introduce those here.

In [28]:
trainer_fn = "nnfabrik.examples.mnist_transfer.trainer.mnist_trainer_fn"

trainer_config = dict(batch_size=64, apply_augmentation=False) 

Trainer().add_entry(trainer_fn=trainer_fn, trainer_config=trainer_config, 
                    trainer_fabrikant="Your Name", trainer_comment="Use Transferred MNIST Logits");

In [29]:
Trainer()

*trainer_fn    *trainer_hash  trainer_co trainer_fabrik trainer_commen trainer_ts    
+------------+ +------------+ +--------+ +------------+ +------------+ +------------+
nnfabrik.examp 79e921430b7f44 =BLOB=     Your Name      A comment abou 2021-01-21 16:
nnfabrik.examp ab91f734757071 =BLOB=     Your Name      Transfer MNIST 2021-01-21 16:
nnfabrik.examp ab91f734757071 =BLOB=     Your Name      Use Transferre 2021-01-21 16:
 (Total: 3)

In [30]:
dataset_fn = "nnfabrik.examples.mnist_transfer.dataset.mnist_dataset_fn"

dataset_config = dict(batch_size=64, apply_augmentation=True) # we specify all the inputs except the ones required by nnfabrik

Dataset().add_entry(dataset_fn=dataset_fn, dataset_config=dataset_config, 
                    dataset_fabrikant="Your Name", dataset_comment="Augmented MNIST with Knowledge Distallation");

In [31]:
Dataset()

*dataset_fn    *dataset_hash  dataset_co dataset_fabrik dataset_commen dataset_ts    
+------------+ +------------+ +--------+ +------------+ +------------+ +------------+
nnfabrik.examp 28aefc23085697 =BLOB=     Your Name      Augmented MNIS 2021-01-21 16:
nnfabrik.examp 9aee736870714f =BLOB=     Your Name      A comment abou 2021-01-21 16:
nnfabrik.examp 28aefc23085697 =BLOB=     Your Name      Augmented MNIS 2021-01-21 16:
 (Total: 3)

In [32]:
transfer_from = {"trainer_fn": 'nnfabrik.examples.mnist_transfer.trainer.mnist_data_gen_fn', 
                 "trainer_hash": 'ab91f734757071bf0b98ab74c6e8583c',
                 "dataset_fn": 'nnfabrik.examples.mnist.dataset.mnist_dataset_fn',
                 "dataset_hash": '9aee736870714f8b7c3cc084087ce886'
                }
transfer_to = {"trainer_fn": 'nnfabrik.examples.mnist_transfer.trainer.mnist_trainer_fn', 
                 "trainer_hash": 'ab91f734757071bf0b98ab74c6e8583c',
                 "dataset_fn": 'nnfabrik.examples.mnist_transfer.dataset.mnist_dataset_fn',
                 "dataset_hash": '28aefc2308569727c6017c66c9122d77'
                }

In [33]:
TrainerDatasetTransferRecipe().add_entry(transfer_from=transfer_from, transfer_to=transfer_to, transfer_step=2, data_transfer=False)

In [34]:
TransferredTrainedModel.transfer_recipe = [TrainerDatasetTransferRecipe()]

### Populate the table for the second transfer step 

In [35]:
TransferredTrainedModel.populate("transfer_step=2")

100%|██████████| 938/938 [00:18<00:00, 50.86it/s]


In [37]:
TransferredTrainedModel()

*transfer_step *model_fn      *model_hash    *dataset_fn    *dataset_hash  *trainer_fn    *trainer_hash  *seed    *collapsed_his *data_transfer comment        score        output     fabrikant_name trainedmodel_t
+------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------+ +------------+ +------------+ +------------+ +----------+ +--------+ +------------+ +------------+
0              nnfabrik.examp 24922759b84307 nnfabrik.examp 9aee736870714f nnfabrik.examp 79e921430b7f44 7                       0              A comment abou 73.7033      =BLOB=     Arne Nix       2021-01-21 16:
1              nnfabrik.examp 24922759b84307 nnfabrik.examp 28aefc23085697 nnfabrik.examp 79e921430b7f44 7        b5dfa65c750917 0              A comment abou 76.1183      =BLOB=     Arne Nix       2021-01-21 16:
1              nnfabrik.examp 24922759b84307 nnfabrik.examp 9aee736870714f nnfabrik.examp ab91f734757071 7        b5dfa65c750917 1              Transfer MNIST 0.0          =BLOB=     Arne Nix       2021-01-21 16:
2              nnfabrik.examp 24922759b84307 nnfabrik.examp 28aefc23085697 nnfabrik.examp ab91f734757071 7        1198fc65667e0a 0              Use Transferre 0.393696     =BLOB=     Arne Nix       2021-01-21 16:
 (Total: 4)